I used the fusion-tutorial from https://adapterhub.ml as a guideline and adjusted it for the fusion of these two adapters adapter: https://adapterhub.ml/adapters/ukp/bert-base-uncased-imdb_pfeiffer/, adapter: https://adapterhub.ml/adapters/ukp/bert-base-uncased_sentiment_sst-2_pfeiffer/



In [ ]:
!pip install -U adapter-transformers
!pip install datasets

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/')

In [ ]:
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"


In [ ]:
from datasets import load_dataset

dataset = load_dataset("sst")
dataset.num_rows

No config specified, defaulting to: sst/default
Reusing dataset sst (/root/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


{'test': 2210, 'train': 8544, 'validation': 1101}

In [ ]:
dataset['train'].features

{'label': Value(dtype='float32', id=None),
 'sentence': Value(dtype='string', id=None),
 'tokens': Value(dtype='string', id=None),
 'tree': Value(dtype='string', id=None)}

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["sentence"], max_length=80, truncation=True, padding="max_length")

dataset = dataset.map(encode_batch, batched=True)

def label_mapping(batch):
  batch["labels"] = round(batch["label"])
  return batch
dataset = dataset.map(label_mapping)

dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

dataset["train"]["labels"]

Loading cached processed dataset at /root/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-4589f93a18dce3fd.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-229cdc0eafef1853.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-9f6af83bc8a11a6b.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-1102ab2b49aad756.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-829c754b2609346c.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/sst/default/1.0.0/b8

tensor([1, 1, 1,  ..., 1, 0, 0])

In [ ]:
from transformers import BertConfig, BertModelWithHeads


config = BertConfig.from_pretrained(
    "bert-base-uncased",
    num_labels=2,


)
model = BertModelWithHeads.from_pretrained(
    "bert-base-uncased",
    config=config,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers.adapters.composition import Fuse

model.load_adapter("sentiment/sst-2@ukp", with_head=False)
model.load_adapter("sentiment/imdb@ukp", with_head=False)
print(model.config.adapters.adapters)


model.add_adapter_fusion(Fuse("sst-2", "imdb"))
model.set_active_adapters(Fuse("sst-2", "imdb"))

model.add_classification_head("sentiment/sst-2@ukp", num_labels=2, id2label={ 0: "👎", 1: "👍"})

Overwriting existing adapter 'sst-2'.
Overwriting existing adapter 'imdb'.


{'sst-2': '9ed5b5a29de19b71', 'imdb': '9076f36a74755ac4'}


In [ ]:
adapter_setup = Fuse("sst-2", "imdb")
model.train_adapter_fusion(adapter_setup)

In [ ]:
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 8544
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1335


Step,Training Loss
200,0.260200
400,0.210900
600,0.229700
800,0.208900
1000,0.195000
1200,0.188600


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/sst-2/adapter_config.json
Module weights saved in ./training_output/checkpoint-500/sst-2/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-500/imdb/adapter_config.json
Module weights saved in ./training_output/checkpoint-500/imdb/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-500/sst-2,rotten_tomatoes/adapter_fusion_config.json
Module weights saved in ./training_output/checkpoint-500/sst-2,rotten_tomatoes/pytorch_model_adapter_fusion.bin
Configuration saved in ./training_output/checkpoint-500/sst-2,imdb/adapter_fusion_config.json
Module weights saved in ./training_output/checkpoint-500/sst-2,imdb/pytorch_model_adapter_fusion.bin
Saving model checkpoint to ./training_output/checkpoint-1000
Configuration saved in ./training_output/checkpoint-1000/sst-2/adapter_config.json
Module weights saved in ./training_output/checkpoint-100

TrainOutput(global_step=1335, training_loss=0.21146062072296715, metrics={'train_runtime': 1476.8635, 'train_samples_per_second': 28.926, 'train_steps_per_second': 0.904, 'total_flos': 2729613647923200.0, 'train_loss': 0.21146062072296715, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1101
  Batch size = 32


{'epoch': 5.0,
 'eval_acc': 0.8628519527702089,
 'eval_loss': 0.38372769951820374,
 'eval_runtime': 16.6681,
 'eval_samples_per_second': 66.054,
 'eval_steps_per_second': 2.1}

In [ ]:
print(model.active_adapters)

model.save_adapter_fusion(path +"models/fusion2/", "sst-2,imdb")
model.save_all_adapters(path +"models/fusion2/")

print(model.active_adapters)




Configuration saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion2/adapter_fusion_config.json


Fuse[sst-2, imdb]


Module weights saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion2/pytorch_model_adapter_fusion.bin
Configuration saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion2/sst-2/adapter_config.json
Module weights saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion2/sst-2/pytorch_adapter.bin
Configuration saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion2/imdb/adapter_config.json
Module weights saved in /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/fusion2/imdb/pytorch_adapter.bin


Fuse[sst-2, imdb]
ls: cannot access 'saved': No such file or directory
